In [ ]:
import math
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.stats import norm
import torch
import matplotlib.pyplot as plt
import distributions as distributions
import options as options
import nn_utils as nn_ut

In [ ]:
# defining the payoff function
K = 1.
option = options.MaxCall(K)

In [ ]:
# setting latex style for plots
plt.rcParams['text.usetex'] = True

In [ ]:
# plotting the payoff function
x_1 = torch.arange(0.75, 1.25, 0.01)
x_2 = torch.arange(0.75, 1.25, 0.01)
xv, yv = torch.meshgrid(x_1, x_2, indexing='ij')
plot_points = torch.stack((xv, yv), dim=2)
zg = option.f(plot_points)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(xv, yv, zg, rstride=1, cstride=1, cmap='coolwarm', edgecolor='none')
plt.show()

In [ ]:
# defining the reference measure
Sigma = torch.tensor([[0.30, 0.], [0.05, 0.20]]) # lower triangular square root of the covariance matrix
t = 1.
mu = distributions.MultiLogNormal(loc=- 0.5 * t * torch.sum(torch.pow(Sigma, 2), dim=1), scale_tril=math.sqrt(t) * Sigma)

samples = mu.sample([100000])
plt.scatter(samples.detach().numpy()[:, 0], samples.detach().numpy()[:, 1])
plt.scatter(torch.exp(- 0.5 * t * torch.sum(torch.pow(Sigma, 2), dim=1))[0], torch.exp(- 0.5 * t * torch.sum(torch.pow(Sigma, 2), dim=1))[1], color='red')
plt.gca().set_aspect('equal')
plt.show()

In [ ]:
# cost functional
p = 3
h = 1. / 12
cost = nn_ut.PowerCost(p=p, h=h, case='second order').cost

In [ ]:
### risk measure by neural network approximation

# fixing the seed
torch.manual_seed(29)

# defining the risk measure object
width = 20
depth = 4
sample_size = 100
risk_measure = nn_ut.MartRiskMeasureMulti(option.f, cost, mu, torch.nn.ReLU, width, depth, d=2)

# otpimizer
optim = torch.optim.Adam(risk_measure.parameters(), lr=0.001)

# training cycle
train_hist = []
epochs = 20000
for i in range(epochs):
    optim.zero_grad()
    y = mu.sample([sample_size])
    risk = risk_measure(y)
    risk.backward()
    optim.step()
    train_hist.append(- float(risk.detach()))

In [ ]:
# plotting the traning phase
roll_window = 200
train_roll = pd.Series(train_hist).rolling(roll_window).mean().dropna()

final_samples = mu.sample([100000])
expected_loss = torch.mean(option.f(final_samples))
plt.plot(1 + np.arange(roll_window, epochs + 1), train_roll, label=f'sup')
plt.axhline(float(expected_loss), color='green', label='expected payoff')
plt.legend()
plt.show()

In [ ]:
# results
risk_measure.eval()

mc_rm = risk_measure(final_samples)
print(f"risk measure mc: {mc_rm:.3e}")
print(f"mc err: {risk_measure.mc_err:.3e}. Mc interval: [{mc_rm - 2.57 * risk_measure.mc_err:.3e}, {mc_rm + 2.57 * risk_measure.mc_err:.3e}]")
print(f"Expected value: {expected_loss:.3e}")

In [ ]:
# contour plots of the loss function and its $C$-transform

x_cont = torch.arange(0.5, 1.5, 0.01)
y_cont = torch.arange(0.5, 1.5, 0.01)
xv, yv = torch.meshgrid(x_cont, y_cont, indexing='ij')
cont_points = torch.stack((xv.flatten(),yv.flatten()), dim=1)

ccong = risk_measure.ccong(cont_points)
orig_func = option.f(cont_points)


# First contour plot
plt.tricontour(cont_points.detach()[:, 0], cont_points.detach()[:, 1], orig_func.detach().flatten(), cmap='plasma')
plt.title('Payoff function')
plt.colorbar()
plt.tight_layout()
plt.show()

# Second contour plot
plt.tricontour(cont_points.detach()[:, 0], cont_points.detach()[:, 1], ccong.detach().flatten(), cmap='plasma')
plt.title('c-transform')
plt.colorbar()
plt.tight_layout()
plt.show()

In [ ]:
# Plot of the C-transform of the loss function

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(xv.detach(), yv.detach(), ccong.reshape(xv.shape).detach(), rstride=1, cstride=1, cmap='coolwarm', edgecolor='none')
plt.tight_layout()
plt.show()